In [1]:
import tensorflow as tf
import pandas as pd

/Users/ale/anaconda3/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
csv_label_name = "medv" # value of the house \$1000s
csv_column_names = ["crim", "zn", "indus", "nox", "rm", "age",
           "dis", "tax", "ptratio", "medv"]
feature_names = ["crim", "zn", "indus", "nox", "rm",
            "age", "dis", "tax", "ptratio"]

train_data = pd.read_csv("datasets/boston-housing/train.csv", names=csv_column_names, skiprows=1, skipinitialspace=True)
test_data = pd.read_csv("datasets/boston-housing/test.csv", names=csv_column_names, skiprows=1, skipinitialspace=True)
predict_data = pd.read_csv("datasets/boston-housing/predict.csv", names=csv_column_names, skiprows=1, skipinitialspace=True)

In [3]:
feature_columns = [
    tf.feature_column.numeric_column(feature) for feature in feature_names
]
    
estimator = tf.estimator.DNNRegressor(hidden_units=[10, 10], feature_columns=feature_columns)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x102a07850>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/var/folders/55/qkrngbr949x1jzs0c0dt70wc0000gn/T/tmpfhAGPg', '_global_id_in_cluster': 0, '_save_summary_steps': 100}


In [4]:
# def input_fn(dataset, epochs=None, shuffle=False):
#     return tf.estimator.inputs.pandas_input_fn(
#         x=pd.DataFrame({ feature: dataset[feature].values for feature in feature_names }),
#         y=pd.Series(dataset[csv_label_name].values),
#         num_epochs=epochs,
#         shuffle=shuffle
#     )

def input_fn(dataset, shuffle=0, batch=128, repeat=False):
    data_set = tf.data.Dataset.from_tensor_slices((dict(dataset), dataset[csv_label_name].values))
    if shuffle > 0:
        data_set = data_set.shuffle(shuffle)
    if repeat:
        data_set = data_set.repeat() # Repeat forever
    return (data_set.batch(batch).make_one_shot_iterator().get_next())

def input_fn_train():
    return input_fn(train_data, repeat=True, shuffle=1000)

def input_fn_test():
    return input_fn(test_data, batch=1)

def input_fn_predict():
    return input_fn(predict_data, batch=1)

estimator.train(input_fn=input_fn_train, steps=5000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/55/qkrngbr949x1jzs0c0dt70wc0000gn/T/tmpfhAGPg/model.ckpt.
INFO:tensorflow:loss = 11108682.0, step = 1
INFO:tensorflow:global_step/sec: 419.169
INFO:tensorflow:loss = 11826.568, step = 101 (0.240 sec)
INFO:tensorflow:global_step/sec: 575.288
INFO:tensorflow:loss = 9481.3955, step = 201 (0.174 sec)
INFO:tensorflow:global_step/sec: 560.243
INFO:tensorflow:loss = 10721.896, step = 301 (0.178 sec)
INFO:tensorflow:global_step/sec: 551.648
INFO:tensorflow:loss = 8160.797, step = 401 (0.181 sec)
INFO:tensorflow:global_step/sec: 559.255
INFO:tensorflow:loss = 12716.093, step = 501 (0.179 sec)
INFO:tensorflow:global_step/sec: 559.018
INFO:tensorflow:loss = 9685.838, step = 601 (0.179 sec)
INFO:tensorflow:

In [5]:
evaluation = estimator.evaluate(input_fn=input_fn_test)
loss_score = evaluation["loss"]
print("Loss: {0:f}".format(loss_score))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-23-23:05:21
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/55/qkrngbr949x1jzs0c0dt70wc0000gn/T/tmpfhAGPg/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-04-23-23:05:21
INFO:tensorflow:Saving dict for global step 5000: average_loss = 27.562271, global_step = 5000, loss = 27.562271
Loss: 27.562271


In [6]:
import itertools
predictions = estimator.predict(input_fn=input_fn_predict)
predictions = list(p["predictions"] for p in itertools.islice(predictions, 6))
print("Predictions: {}".format(str(predictions)))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/55/qkrngbr949x1jzs0c0dt70wc0000gn/T/tmpfhAGPg/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Predictions: [array([31.348598], dtype=float32), array([17.069601], dtype=float32), array([26.046022], dtype=float32), array([31.226858], dtype=float32), array([15.401953], dtype=float32), array([19.919788], dtype=float32)]
